In [1]:
from tkinter import * 
from tkinter import filedialog,messagebox
import cv2
import os
import numpy as np
import pandas as pd
import csv
import scipy.stats
import math

In [2]:
from keras import applications
import keras.backend as K
from keras.optimizers import Adam
from keras.models import Model
from keras.models import load_model
from keras.layers import Input,concatenate

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
img_width, img_height, channels = 224, 224, 3
dim = (img_width, img_height)

In [4]:
model = applications.vgg19.VGG19(weights='imagenet', include_top=False, pooling='max')
for layer in model.layers[:15]:
    layer.trainable = False

In [5]:
anchor_in = Input(shape=(img_width, img_height, channels))
pos_in = Input(shape=(img_width, img_height, channels))
neg_in = Input(shape=(img_width, img_height, channels))

anchor_out = model(anchor_in)
pos_out = model(pos_in)
neg_out = model(neg_in)
merged_vector = concatenate([anchor_out, pos_out, neg_out],axis=1)

model = Model(inputs=[anchor_in, pos_in, neg_in], outputs=merged_vector)

In [6]:
model.load_weights('C:/Users/INDIA/Desktop/axis bank hack/model_weights.h5')

In [7]:
def to_rgb(img):
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
    img_rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    return img_rgb

def returnimages(path):
    image=cv2.imread(path)                  #bringing the image
    image=cv2.resize(image, (img_width, img_height))
    image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image=to_rgb(image).reshape(1,img_width, img_height,3)/255.0       #resizing and normalizing    
    return image



In [68]:
dirc="C:/Users/INDIA/Desktop"
os.listdir(dirc)
fol1=dirc+"/DataBase"
fol2=dirc+"/Test"

In [13]:
os.makedirs(dirc+"/bankdetails/")

In [14]:
dir_bankdetails=dirc+"/bankdetails/"

In [67]:
listofcust=os.listdir(fol1)
listofcheck=os.listdir(fol2)
#listofcust[0]

In [42]:
for i in range(0,len(listofcust),3):
   # print(listofcust[i],listofcust[i+1],listofcust[i+2])
    anc1=fol1+"/"+listofcust[i]
    anc2=fol1+"/"+listofcust[i+1]
    anc3=fol1+"/"+listofcust[i+2]
    
    anchor1=returnimages(anc1)
    anchor2=returnimages(anc2)
    anchor3=returnimages(anc3)   

    x=model.predict([anchor1,anchor2,anchor3])

    a1, a2, a3 = x[0,0:512], x[0,512:1024], x[0,1024:1536]

    thresh1=np.linalg.norm(a1-a2)
    thresh2=np.linalg.norm(a2-a3)
    thresh3=np.linalg.norm(a1-a3)
    thresh=(thresh1+thresh2+thresh3)/3
    thresh=thresh+3.5 #adjust the adding parameter
    thresh=[thresh]
    
        
    y=dir_bankdetails+listofcust[i][0:6]+"_1"+".csv"
    columns="columns"
    np.savetxt(y,a1, header=columns,comments="")

    
    index=listofcust[i].find('_')
    
    y=dir_bankdetails+listofcust[i][0:6]+"_2"+".csv"
    columns="columns"
    np.savetxt(y,a2, header=columns,comments="")

    y=dir_bankdetails+listofcust[i][0:6]+"_3"+".csv"
    columns="columns"
    np.savetxt(y,a3, header=columns,comments="")

    y=dir_bankdetails+listofcust[i][0:6]+"_thresh"+".csv"
    columns="thresh"
    np.savetxt(y,thresh, header=columns,comments="")
    
    
    

U11001_152.jpg U11001_188.jpg U11001_79.jpg
U11002_107.jpg U11002_132.jpg U11002_233.jpg
U11003_103.jpg U11003_308.jpg U11003_9.jpg
U11004_218.jpg U11004_346.jpg U11004_393.jpg
U11005_361.jpg U11005_452.jpg U11005_85.jpg
U11006_11.jpg U11006_253.jpg U11006_96.jpg
U11007_300.jpg U11007_381.jpg U11007_4.jpg
U11008_198.jpg U11008_235.jpg U11008_374.jpg
U11009_234.jpg U11009_274.jpg U11009_299.jpg
U11010_173.jpg U11010_225.jpg U11010_478.jpg
U11011_14.jpg U11011_18.jpg U11011_263.jpg
U11012_171.jpg U11012_261.jpg U11012_368.jpg
U11013_164.jpg U11013_455.jpg U11013_98.jpg
U11014_190.jpg U11014_199.jpg U11014_383.jpg
U11015_231.jpg U11015_44.jpg U11015_449.jpg
U11016_135.jpg U11016_33.jpg U11016_431.jpg
U11017_129.jpg U11017_159.jpg U11017_324.jpg
U11018_187.jpg U11018_27.jpg U11018_399.jpg
U11019_319.jpg U11019_354.jpg U11019_86.jpg
U11020_257.jpg U11020_480.jpg U11020_67.jpg
U11021_317.jpg U11021_357.jpg U11021_75.jpg
U11024_251.jpg U11024_28.jpg U11024_434.jpg
U11025_102.jpg U11025_284.jp

In [96]:
nameoffiles=[]
decision=[]

In [97]:
for i in range(0,len(listofcheck)):
    nameoffiles.append(listofcheck[i])

In [95]:
#getting anchors and threshold from csv, copy paste method.
#!!!!!!!!!!!
#csv not created
#!!!!!!!!!!!
for i in range(0,len(listofcheck)):
    tst=fol2+"/"+listofcheck[i]
    name=listofcheck[i][0:6]
    
    a=pd.read_csv(dir_bankdetails+name+"_1"+".csv")
    o1=np.array([])
    for q in range(512):
        o1=np.append(o1,a.loc[q])
    
    a=pd.read_csv(dir_bankdetails+name+"_2"+".csv")
    o2=np.array([])
    for q in range(512):
        o2=np.append(o2,a.loc[q])
        
    a=pd.read_csv(dir_bankdetails+name+"_3"+".csv")
    o3=np.array([])
    for q in range(512):
        o3=np.append(o3,a.loc[q])
        
    a=pd.read_csv(dir_bankdetails+name+"_thresh"+".csv")
    thr=np.array([])
    for q in range(a.shape[0]):
        thr=np.append(thr,a.loc[q])
    
    #tst=fol2+"/"+listofcheck[i]
    imag=returnimages(tst)
    #print(tst)
    x=model.predict([imag,imag,imag])
    t1 = x[0,0:512]
    dist1=np.linalg.norm(o1-t1)
    dist2=np.linalg.norm(o2-t1)
    dist3=np.linalg.norm(o3-t1)
    
    dist=(dist1+dist2+dist3)/3
    
    if(dist>thr):
        print("No")
    else:
        print("Yes")

Yes
Yes
No
Yes
Yes
No
Yes
No
No
Yes
No
No
No
Yes
No
No
No
No
No
Yes
Yes
No
No
No
No
No
Yes
No
No
Yes
No
No
No
No
No
No
Yes
Yes
Yes
Yes
No
Yes
No
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
No
Yes
No
No
No
No
No
No
No
No
No
Yes
No
No
Yes
No
Yes
No
Yes
No
No
No
No
Yes
No
No
No
No
Yes
Yes
No
No
No
Yes
No
No
No
No
Yes
No
Yes
Yes
No
No
No
Yes
No
No
No
Yes
No
No
No
No
No
Yes
No
No
No
No
No
Yes
Yes
No
No
No
Yes
Yes
No
No
Yes
Yes
No
Yes
No
No
No
No
No
Yes
Yes
Yes
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
No
Yes
No
Yes
No
No
No
No
Yes
No
No
No
No
Yes
No
Yes
Yes
No
Yes
Yes
Yes
No
No
No
No
No
Yes
No
No
No
No
No
No
Yes
Yes
No
No
Yes
Yes
Yes
No
Yes
Yes
No
No
No
No
No
No
No
No
No
No
Yes
No
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
No
Yes
No
No
Yes
Yes
Yes
No
Yes
No
Yes
No
No
No
No
Yes
Yes
Yes
Yes
No
No
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
No
No
No
No
No
No
No
No
No
Yes
No
No
Yes
No
No
Yes
No
Yes
Yes
No
Yes
Yes
No
Yes
No
No
No
Yes
No
Yes
No
No
No
No
No
No
No
No
No
No
No
No
No
No
Yes
Ye

In [99]:
#getting anchors and threshold from csv and comparing it with test images
for i in range(0,len(listofcheck)):
    tst=fol2+"/"+listofcheck[i]
    name=listofcheck[i][0:6]
    
    a=pd.read_csv(dir_bankdetails+name+"_1"+".csv")
    o1=np.array([])
    for q in range(512):
        o1=np.append(o1,a.loc[q])
    
    a=pd.read_csv(dir_bankdetails+name+"_2"+".csv")
    o2=np.array([])
    for q in range(512):
        o2=np.append(o2,a.loc[q])
        
    a=pd.read_csv(dir_bankdetails+name+"_3"+".csv")
    o3=np.array([])
    for q in range(512):
        o3=np.append(o3,a.loc[q])
        
    a=pd.read_csv(dir_bankdetails+name+"_thresh"+".csv")
    thr=np.array([])
    for q in range(a.shape[0]):
        thr=np.append(thr,a.loc[q])
    
    #tst=fol2+"/"+listofcheck[i]
    imag=returnimages(tst)
    #print(tst)
    x=model.predict([imag,imag,imag])
    t1 = x[0,0:512]
    dist1=np.linalg.norm(o1-t1)
    dist2=np.linalg.norm(o2-t1)
    dist3=np.linalg.norm(o3-t1)
    
    dist=(dist1+dist2+dist3)/3
    
    if(dist>thr):
        print("No")
        decision.append("No")
    else:
        print("Yes")
        decision.append("Yes")

Yes
Yes
No
Yes
Yes
No
Yes
No
No
Yes
No
No
No
Yes
No
No
No
No
No
Yes
Yes
No
No
No
No
No
Yes
No
No
Yes
No
No
No
No
No
No
Yes
Yes
Yes
Yes
No
Yes
No
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
No
Yes
No
No
No
No
No
No
No
No
No
Yes
No
No
Yes
No
Yes
No
Yes
No
No
No
No
Yes
No
No
No
No
Yes
Yes
No
No
No
Yes
No
No
No
No
Yes
No
Yes
Yes
No
No
No
Yes
No
No
No
Yes
No
No
No
No
No
Yes
No
No
No
No
No
Yes
Yes
No
No
No
Yes
Yes
No
No
Yes
Yes
No
Yes
No
No
No
No
No
Yes
Yes
Yes
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
No
Yes
No
Yes
No
No
No
No
Yes
No
No
No
No
Yes
No
Yes
Yes
No
Yes
Yes
Yes
No
No
No
No
No
Yes
No
No
No
No
No
No
Yes
Yes
No
No
Yes
Yes
Yes
No
Yes
Yes
No
No
No
No
No
No
No
No
No
No
Yes
No
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
No
Yes
No
No
Yes
Yes
Yes
No
Yes
No
Yes
No
No
No
No
Yes
Yes
Yes
Yes
No
No
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
No
No
No
No
No
No
No
No
No
Yes
No
No
Yes
No
No
Yes
No
Yes
Yes
No
Yes
Yes
No
Yes
No
No
No
Yes
No
Yes
No
No
No
No
No
No
No
No
No
No
No
No
No
No
Yes
Ye

In [100]:
with open(dirc+'/solutionnew.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(zip(["Filename"],["Forgeries"]))
    writer.writerows(zip(nameoffiles, decision))